____
__Universidad de San Andrés__<br/>
__Machine Learning__<br/>
__Modelado probabilistico de audios de ballenas y estudio de transfer learning__<br/>
__Martin Bianchi y Federico Gutman__
____

### Importamos las librerías necesarias

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
from IPython.display import Audio, display
from sklearn.decomposition import PCA 

### NOTAS
- ver repo para preprocesar espectrogrma
    - ver de reducir el y-lim del spectrograma -> averiguar bien entre que frecuencias se mueven los cantos de ballena
- cambiar noise a no right whale
- cambiar el idioma a español


### Definimos algunas variables globales

In [ ]:
SEED = 3            # semilla para reproductibilidad
SR = 2000           # sampling rate
N_FFT = 256         # tamaño de la ventana
HOP_LENGTH = 64     # salto entre frames
N_MELS = 50         # frequency bins (resolución)
MAX_FREQ = 600      # máxima frecuencia para los espectrogramas --ponerlo en 500--

### Cargamos los datos y los visualizamos

In [ ]:
def load_data(train_path:str, test_path:str, labels_path:str, sampling_rate=SR):
    test_files = [f for f in os.listdir(test_path) if f.endswith('.aiff')]
    labels_df = pd.read_csv(labels_path)

    audio_df = labels_df.copy()
    audio_df['filepath'] = audio_df['clip_name'].apply(lambda x: os.path.join(train_path, x))
    audio_df['audio'] = audio_df['filepath'].apply(lambda path: librosa.load(path, sr=sampling_rate)[0])

    return audio_df, labels_df, test_files

def display_random_samples(dataset:pd.DataFrame, sampling_rate=SR):
    np.random.seed(SEED)

    # SAMPLES
    whale_sample = dataset[dataset['label'] == 1].sample(5)['audio']
    noise_sample = dataset[dataset['label'] == 0].sample(5)['audio']

    # AUDIO
    print('right whale call random audio sample')
    display(Audio(np.array(whale_sample.iloc[0]), rate=sampling_rate*1.5))
    print('\nno whale random audio sample')
    display(Audio(np.array(noise_sample.iloc[0]), rate=sampling_rate*1.5))

    # SOUND-WAVE
    fig, axes = plt.subplots(2, 5, figsize=(15, 5))

    for i in range(5):
        axes[0, i].plot(np.array(whale_sample.iloc[i]))
        axes[0, i].set_title('Whale')
        axes[0, i].set_xticks([])
        axes[0, i].set_yticks([])

        axes[1, i].plot(np.array(noise_sample.iloc[i]), color='#FF6961')
        axes[1, i].set_title('No Whale')
        axes[1, i].set_xticks([])
        axes[1, i].set_yticks([])

    plt.tight_layout()
    plt.show()

    # MULTIPLE SPECTROGRAMS
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))

    for i in range(5):
        whale_sample_spectrogram = get_melspectrogram(whale_sample.iloc[i])
        img0 = librosa.display.specshow(whale_sample_spectrogram, sr=sampling_rate, hop_length=HOP_LENGTH, ax=axes[0, i], x_axis='time', y_axis='hz', cmap='magma', fmax=MAX_FREQ)
        axes[0, i].set_title('Whale')
        axes[0, i].set_ylim([0, MAX_FREQ])
        axes[0, i].set_xticks([])
        axes[0, i].set_yticks([])
        axes[0, i].set_xlabel('')
        axes[0, i].set_ylabel('')

        noise_sample_spectrogram = get_melspectrogram(noise_sample.iloc[i])
        img1 = librosa.display.specshow(noise_sample_spectrogram, sr=sampling_rate, hop_length=HOP_LENGTH, ax=axes[1, i], x_axis='time', y_axis='hz', cmap='magma', fmax=MAX_FREQ)
        axes[1, i].set_title('No Whale')
        axes[1, i].set_ylim([0, MAX_FREQ])
        axes[1, i].set_xticks([])
        axes[1, i].set_yticks([])
        axes[1, i].set_xlabel('')
        axes[1, i].set_ylabel('')

    plt.tight_layout()
    plt.show()

    # TWO SPECTROGRAMS
    whale_sample2 = dataset[dataset['label'] == 1].sample(1, random_state=SEED+1)['audio']
    noise_sample2 = dataset[dataset['label'] == 0].sample(1, random_state=SEED+1)['audio']
    
    fig, axes = plt.subplots(1, 2, figsize=(7, 3))

    whale_sample2_spectrogram = get_melspectrogram(whale_sample2.iloc[0])
    img0 = librosa.display.specshow(whale_sample2_spectrogram, sr=sampling_rate, hop_length=HOP_LENGTH, ax=axes[0], x_axis='time', y_axis='hz', cmap='magma', fmax=MAX_FREQ)
    axes[0].set_title('Whale')
    axes[0].set_ylim([0, MAX_FREQ])
    fig.colorbar(img0, ax=axes[0], format="%+2.0f dB")

    noise_sample2_spectrogram = get_melspectrogram(noise_sample2.iloc[0])
    img1 = librosa.display.specshow(noise_sample2_spectrogram, sr=sampling_rate, hop_length=HOP_LENGTH, ax=axes[1], x_axis='time', y_axis='hz', cmap='magma', fmax=MAX_FREQ)
    axes[1].set_title('No Whale')
    axes[1].set_ylim([0, MAX_FREQ])
    fig.colorbar(img1, ax=axes[1], format="%+2.0f dB")

    plt.tight_layout()
    plt.show()

def normalize(dataset:pd.DataFrame, column:str):
    dataset[column] = dataset[column].apply(lambda x: x / np.max(np.abs(x)))

def get_signal_energy(dataset:pd.DataFrame, column:str):
    energy = []
    df = dataset.copy()
    for x in df[column]:
        energy.append(np.sum(np.square(x)))
    df['energy'] = energy

    return df

def get_melspectrogram(sample:pd.DataFrame, sampling_rate=SR, n_fft=N_FFT, hop_length=HOP_LENGTH, n_mels=N_MELS):
    mel_spectrogram = librosa.feature.melspectrogram(y=np.array(sample), sr=sampling_rate, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, fmax=MAX_FREQ)
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
    return mel_spectrogram

def get_all_mel_spectrograms(audio_df, sampling_rate=SR, n_fft=N_FFT, hop_length=HOP_LENGTH, n_mels=N_MELS):
    mel_specs = []
    for audio in audio_df['audio']:
        mel = get_melspectrogram(audio, sampling_rate, n_fft, hop_length, n_mels)
        mel_specs.append(mel.flatten())
    return np.array(mel_specs)

def show_class_balance(dataset:pd.DataFrame):
    plt.figure(figsize=(5, 4))
    ax = sns.countplot(
        x='label',
        data=dataset,
        palette={0: '#FF6961', 1: plt.rcParams['axes.prop_cycle'].by_key()['color'][0]},
        hue='label',
        legend=False
    )
    ax.set_xticks([0, 1])
    ax.set_xticklabels(['Noise', 'Whale'])
    ax.legend(['Noise', 'Whale'], title='Class')
    plt.show()

def extract_time_acoustic_features(dataset:pd.DataFrame):
    new_features = []
    for audio in dataset['audio']:
        audio_np = np.array(audio)
        acoustic_features = {
            'rms_energy': np.mean(librosa.feature.rms(y=audio_np)),
            'zcr': np.mean(librosa.feature.zero_crossing_rate(y=audio_np)),
        }
        new_features.append(acoustic_features)
    features_df = pd.DataFrame(new_features)
    return features_df

def extract_frequency_acoustic_features(dataset:pd.DataFrame):
    new_features = []
    for audio in dataset['audio']:
        audio_np = np.array(audio)
        acoustic_features = {
            'spectral_centroid': np.mean(librosa.feature.spectral_centroid(y=audio_np)),
            'spectral_bandwidth': np.mean(librosa.feature.spectral_bandwidth(y=audio_np)),
            'spectral_rolloff': np.mean(librosa.feature.spectral_rolloff(y=audio_np)),
            'spectral_flatness': np.mean(librosa.feature.spectral_flatness(y=audio_np))
        }
        new_features.append(acoustic_features)
    features_df = pd.DataFrame(new_features)
    return features_df

def extract_acoustic_features(dataset:pd.DataFrame):
    time_features = extract_time_acoustic_features(dataset)
    frequency_features = extract_frequency_acoustic_features(dataset)

    new_df = pd.concat([dataset.reset_index(drop=True), time_features], axis=1)
    new_df = pd.concat([new_df.reset_index(drop=True), frequency_features], axis=1)
    
    return new_df

def display_features_boxplots(audio_features_df:pd.DataFrame):
    time_features = ['rms_energy', 'zcr']
    freq_features = ['spectral_centroid', 'spectral_bandwidth', 'spectral_rolloff', 'spectral_flatness']

    # TIME
    fig, axes = plt.subplots(1, len(time_features), figsize=(5 * len(time_features), 5))
    if len(time_features) == 1:
        axes = [axes]
    for i, feature in enumerate(time_features):
        sns.boxplot(x='label', y=feature, data=audio_features_df, ax=axes[i], showfliers=False)
        axes[i].set_title(f'{feature} by Class')
        axes[i].set_xlabel('Class')
        axes[i].set_xticks([0, 1])
        axes[i].set_xticklabels(['Noise', 'Whale'])

    plt.suptitle('Time Domain Features by Class')
    plt.tight_layout()
    plt.show()

    # FREQUENCY
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    axes = axes.flatten()
    for i, feature in enumerate(freq_features):
        sns.boxplot(x='label', y=feature, data=audio_features_df, ax=axes[i], showfliers=False)
        axes[i].set_title(f'{feature} by Class')
        axes[i].set_xlabel('Class')
        axes[i].set_xticks([0, 1])
        axes[i].set_xticklabels(['Noise', 'Whale'])

    plt.suptitle('Frequency Domain Features by Class')
    plt.tight_layout()
    plt.show()

def plot_pca(dataset:pd.DataFrame, pca_arr):
    plt.figure(figsize=(8,6))
    scatter = sns.scatterplot(
        x=pca_arr[:,0], 
        y=pca_arr[:,1], 
        hue=dataset['label'], 
        palette={0: '#FF6961', 1: '#1f77b4'}
    )
    plt.title('PCA of Mel Spectrograms')
    plt.xlabel('PC 1')
    plt.ylabel('PC 2')
    # Fix legend labels and colors
    handles, _ = scatter.get_legend_handles_labels()
    scatter.legend(handles=handles, title='Class', labels=['Noise', 'Whale'])
    plt.show()

def plot_average_spectrograms(audio_df:pd.DataFrame):
    whale_spectrograms = []
    noise_spectrograms = []
    for audio, label in zip(audio_df['audio'], audio_df['label']):
        spectrogram = get_melspectrogram(audio)
        if label == 1:
            whale_spectrograms.append(spectrogram)
        else: 
            noise_spectrograms.append(spectrogram)
    
    whale_spectrograms = np.array(whale_spectrograms)
    noise_spectrograms = np.array(noise_spectrograms)

    whale_average_spectrogram = whale_spectrograms.mean(axis=0)
    noise_average_spectrogram = noise_spectrograms.mean(axis=0)
    average_differences_spectrogram = whale_average_spectrogram - noise_average_spectrogram

    fig, axes = plt.subplots(1, 3, figsize=(10, 3))
    img0 = librosa.display.specshow(whale_average_spectrogram, sr=SR, hop_length=HOP_LENGTH, ax=axes[0], x_axis='time', y_axis='hz', cmap='magma', fmax=MAX_FREQ)
    axes[0].set_title('Whale Call Average Spectrogram', pad=20)
    axes[0].set_ylim([0, MAX_FREQ])
    # axes[0].set_xticks([])
    # axes[0].set_yticks([])
    fig.colorbar(img0, ax=axes[0], format="%+2.0f dB")

    img1 = librosa.display.specshow(noise_average_spectrogram, sr=SR, hop_length=HOP_LENGTH, ax=axes[1], x_axis='time', y_axis='hz', cmap='magma', fmax=MAX_FREQ)
    axes[1].set_title('No Whale Call Average Spectrogram', pad=20)
    axes[1].set_ylim([0, MAX_FREQ])
    # axes[1].set_xticks([])
    # axes[1].set_yticks([])
    fig.colorbar(img1, ax=axes[1], format="%+2.0f dB")

    img2 = librosa.display.specshow(average_differences_spectrogram, sr=SR, hop_length=HOP_LENGTH, ax=axes[2], x_axis='time', y_axis='hz', cmap='magma', fmax=MAX_FREQ)
    axes[2].set_title('Average Differences Spectrogram', pad=20)
    axes[2].set_ylim([0, MAX_FREQ])
    # axes[2].set_xticks([])
    # axes[2].set_yticks([])
    fig.colorbar(img2, ax=axes[2], format="%+2.0f dB")

    # 3 subplots -> average spectrogram whale, noise, resto y consigo diferencias
    plt.tight_layout()
    plt.show()


In [ ]:
train_dir = 'data/whale-detection-challenge/data/train'
test_dir = 'data/whale-detection-challenge/data/test'
labels_dir = 'data/whale-detection-challenge/data/train.csv'

audio_df, labels_df, test_files = load_data(train_dir, test_dir, labels_dir)

In [ ]:
normalize(audio_df, 'audio')
display_random_samples(audio_df)

### Hacemos un análisis exploratorio de los datos

In [ ]:
show_class_balance(audio_df)

In [ ]:
plot_average_spectrograms(audio_df)

In [ ]:
audio_features_df = extract_acoustic_features(audio_df)
display_features_boxplots(audio_features_df)

In [ ]:
# EXPERIMENTO PCA
flattened_mel_spectrogram = get_all_mel_spectrograms(audio_df)
pca = PCA(n_components=2)
spec_pca = pca.fit_transform(flattened_mel_spectrogram)

plot_pca(audio_df, spec_pca)

### Arrancamos el modelado

### Importamos las librerías necesarias

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, roc_curve, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

In [63]:
def normalize_spectrogram(spec, global_min, global_max):
    norm_spec = (spec - global_min) / (global_max - global_min + 1e-8)
    return norm_spec

def denormalize_spectrogram(norm_spec, min_val, max_val):
    return norm_spec * (max_val - min_val + 1e-8) + min_val

def compute_global_min_max(audio_df):
    whale_specs = [get_melspectrogram(audio) for audio, label in zip(audio_df['audio'], audio_df['label']) if label == 1]
    global_min = np.min([spec.min() for spec in whale_specs])
    global_max = np.max([spec.max() for spec in whale_specs])
    return global_min, global_max

def get_class_weights(y):
    weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
    return torch.tensor(weights, dtype=torch.float32)

def get_data_loaders(train_df):
    X = get_all_mel_spectrograms(train_df)
    y = train_df['label'].values
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long)

    train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=128, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=128)

    return train_loader, val_loader



### Creamos las arquitecturas necesarias para el clasificador

In [ ]:
# ------------- MLP -------------

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_layers=[256, 128, 64], output_dim=2):
        super(MLP, self).__init__()
        layers = []
        prev_dim = input_dim

        for hidden_layer in hidden_layers:
            layers.append(nn.Linear(prev_dim, hidden_layer))
            layers.append(nn.ReLU())
            prev_dim = hidden_layer
        layers.append(nn.Linear(prev_dim, output_dim))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)
    
    def train_model(self, train_loader, val_loader, epochs=50, lr=0.001, weight_decay=1e-5):
        self.to(self.device)
        y_train = train_loader.dataset.tensors[1].cpu().numpy().flatten()
        criterion = nn.CrossEntropyLoss(weight=get_class_weights(y_train).to(self.device))
        optimizer = optim.Adam(self.parameters(), lr=lr, weight_decay=weight_decay)

        for epoch in range(epochs):
            self.train()
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                optimizer.zero_grad()
                outputs = self(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            self.evaluate(val_loader)
            print(f'Epoch {epoch+1}/{epochs}, Train Loss: {loss.item():.4f}')
    
    def evaluate(self, val_loader):
        self.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self(inputs)
                loss = nn.CrossEntropyLoss()(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        accuracy = 100 * correct / total
        print(f'Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%')

# ESTO DE ABAJO NO ANDA TODAVIA PERO ES PARA TENER
def MLP_cross_val(train_loader:DataLoader, val_loader:DataLoader, input_dim:int, hidden_layers:list, output_dim:int, epochs:int, lr:list, weight_decay:list, regularization_term:list, early_stopping_patience:list):
    best_model = None
    best_val_loss = float('inf')
    best_params = None
    global_min, global_max = compute_global_min_max(train_loader.dataset)
    for lr_val in lr:
        for wd in weight_decay:
            for reg in regularization_term:
                model = MLP(input_dim, hidden_layers, output_dim)
                model.train(train_loader, val_loader, epochs=epochs, lr=lr_val, weight_decay=wd)

                # Evaluate on validation set
                val_loss = 0.0
                model.eval()
                with torch.no_grad():
                    for inputs, labels in val_loader:
                        inputs, labels = inputs.to(model.device), labels.to(model.device)
                        outputs = model(inputs)
                        loss = nn.CrossEntropyLoss()(outputs, labels)
                        val_loss += loss.item()

                val_loss /= len(val_loader)

                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    best_model = model
                    best_params = (lr_val, wd, reg)
    print(f'Best Validation Loss: {best_val_loss:.4f} with params: LR={best_params[0]}, WD={best_params[1]}, Reg={best_params[2]}')
    return best_model, best_params    

# -------------------------------

### Obtenemos los audios de entrenamiento

In [ ]:
audio_train_df = audio_df[audio_df['clip_name'].str.contains('train')]

### Entrenamos alternando entre una loss con pesos y cambiando la máxima frecuencia de los espectrogramas

In [ ]:
# nota: aca estamos overfitteando -> hay que separar en folds

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

original_max_freq = MAX_FREQ
for mf in [500, 600, 1000]:
    print(f'\n--- MAX_FREQ = {mf} HZ ---')

    MAX_FREQ = mf
    train_loader, val_loader = get_data_loaders(audio_train_df)
    X_train = train_loader.dataset.tensors[0].cpu().numpy()
    y_train = train_loader.dataset.tensors[1].cpu().numpy().flatten()

    for weighted in [False, True]:
        print(f"\n--- WEIGHTED LOSS = {weighted} ---")
        
        mlp = MLP(X_train.shape[1]).to(device)
        if weighted:
            weights = get_class_weights(y_train).to(device)
            criterion = nn.CrossEntropyLoss(weight=weights)
        else:
            criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(mlp.parameters(), lr=1e-3)

        best_auc = 0
        patience = 3
        counter = 0

        for epoch in range(20):
            mlp.train()
            for xb, yb in train_loader:
                xb, yb = xb.to(device), yb.to(device)
                optimizer.zero_grad()
                loss = criterion(mlp(xb), yb)
                loss.backward()
                optimizer.step()

            mlp.eval()
            all_probs, all_targets = [], []
            with torch.no_grad():
                for xb, yb in val_loader:
                    xb = xb.to(device)
                    probs = torch.softmax(mlp(xb), dim=1)[:, 1].cpu().numpy()
                    all_probs.extend(probs)
                    all_targets.extend(yb.numpy())
            auc = roc_auc_score(all_targets, all_probs)
            print(f"Epoch {epoch+1} - AUC: {auc:.4f}")

            if auc > best_auc:
                best_auc = auc
                counter = 0
            else:
                counter += 1
                if counter >= patience:
                    print("Early stopping")
                    break

        print(f"AUC MLP final: {best_auc:.4f}")
        
        y_pred = [1 if p > 0.5 else 0 for p in all_probs]
        f1 = f1_score(all_targets, y_pred)
        print(f"F1-SCORE: {f1:.4f}")

MAX_FREQ = original_max_freq


--- MAX_FREQ = 500 HZ ---

--- WEIGHTED LOSS = False ---
Epoch 1 - AUC: 0.9143
Epoch 2 - AUC: 0.9206
Epoch 3 - AUC: 0.9266
Epoch 4 - AUC: 0.9349
Epoch 5 - AUC: 0.9384
Epoch 6 - AUC: 0.9354
Epoch 7 - AUC: 0.9389
Epoch 8 - AUC: 0.9374
Epoch 9 - AUC: 0.9432
Epoch 10 - AUC: 0.9436
Epoch 11 - AUC: 0.9426
Epoch 12 - AUC: 0.9438
Epoch 13 - AUC: 0.9408
Epoch 14 - AUC: 0.9394
Epoch 15 - AUC: 0.9424
Early stopping
AUC MLP final: 0.9438
F1-SCORE: 0.7570

--- WEIGHTED LOSS = True ---
Epoch 1 - AUC: 0.9050
Epoch 2 - AUC: 0.9166
Epoch 3 - AUC: 0.9270
Epoch 4 - AUC: 0.9266
Epoch 5 - AUC: 0.9350
Epoch 6 - AUC: 0.9356
Epoch 7 - AUC: 0.9363
Epoch 8 - AUC: 0.9385
Epoch 9 - AUC: 0.9356
Epoch 10 - AUC: 0.9397
Epoch 11 - AUC: 0.9441
Epoch 12 - AUC: 0.9456
Epoch 13 - AUC: 0.9431
Epoch 14 - AUC: 0.9392
Epoch 15 - AUC: 0.9461
Epoch 16 - AUC: 0.9466
Epoch 17 - AUC: 0.9411
Epoch 18 - AUC: 0.9476
Epoch 19 - AUC: 0.9467
Epoch 20 - AUC: 0.9459
AUC MLP final: 0.9476
F1-SCORE: 0.7638

--- MAX_FREQ = 600 HZ ---

--- 

### Preprocesamos los datos para dárselos al modelo
weight y maxfreq no cambio casi nada, habria q entender pq 

In [72]:

MAX_FREQ = 500

X = get_all_mel_spectrograms(audio_train_df)
y = audio_train_df['label'].values

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Convert numpy arrays to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# Create DataLoaders
batch_size = 128
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=batch_size)

KeyboardInterrupt: 

### Entrenamos 3 clasificadores


In [ ]:
# --------------------------
# MLP (PyTorch)
# --------------------------
mlp_torch = MLP(X_train.shape[1])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mlp_torch.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp_torch.parameters(), lr=1e-3)

epochs = 20
for epoch in range(epochs):
    mlp_torch.train()
    running_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        outputs = mlp_torch(xb)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * xb.size(0)
    avg_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

    mlp_torch.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            outputs = mlp_torch(xb)
            _, predicted = torch.max(outputs, 1)
            total += yb.size(0)
            correct += (predicted == yb).sum().item()
    val_acc = correct / total
    print(f"Validation Accuracy (MLP): {val_acc:.4f}")

# --------------------------
# Random Forest
# --------------------------
def train_random_forest(X_train, y_train, X_val, y_val):
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    preds = rf.predict(X_val)
    acc = accuracy_score(y_val, preds)
    print(f"Validation Accuracy (Random Forest): {acc:.4f}")
    return rf

# --------------------------
# Gradient Boosting
# --------------------------
def train_gradient_boosting(X_train, y_train, X_val, y_val):
    gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
    gb.fit(X_train, y_train)
    preds = gb.predict(X_val)
    acc = accuracy_score(y_val, preds)
    print(f"Validation Accuracy (Gradient Boosting): {acc:.4f}")
    return gb

# --------------------------
# Ejecutar modelos sklearn
# --------------------------
# Convertimos a numpy si venís usando tensores
X_train_np = X_train_tensor.numpy()
y_train_np = y_train_tensor.numpy()
X_val_np = X_val_tensor.numpy()
y_val_np = y_val_tensor.numpy()

rf_model = train_random_forest(X_train_np, y_train_np, X_val_np, y_val_np)
gb_model = train_gradient_boosting(X_train_np, y_train_np, X_val_np, y_val_np)


### Visualizamos

In [ ]:


# Asegurate de que y_val_np esté definido
# (ya lo tenés si ejecutaste los bloques anteriores)
# y que las etiquetas estén en formato binario (0/1)

# 1. MLP - predicción de probabilidades
mlp_torch.eval()
mlp_probs = []
with torch.no_grad():
    for xb, _ in val_loader:
        xb = xb.to(device)
        outputs = mlp_torch(xb)
        probs = F.softmax(outputs, dim=1)[:, 1]  # clase positiva
        mlp_probs.extend(probs.cpu().numpy())
mlp_probs = np.array(mlp_probs)
mlp_auc = roc_auc_score(y_val_np, mlp_probs)
fpr_mlp, tpr_mlp, _ = roc_curve(y_val_np, mlp_probs)

# 2. Random Forest
rf_probs = rf_model.predict_proba(X_val_np)[:, 1]
rf_auc = roc_auc_score(y_val_np, rf_probs)
fpr_rf, tpr_rf, _ = roc_curve(y_val_np, rf_probs)

# 3. Gradient Boosting
gb_probs = gb_model.predict_proba(X_val_np)[:, 1]
gb_auc = roc_auc_score(y_val_np, gb_probs)
fpr_gb, tpr_gb, _ = roc_curve(y_val_np, gb_probs)

# 4. Plot all ROC curves
plt.figure(figsize=(8, 6))
plt.plot(fpr_mlp, tpr_mlp, label=f'MLP (AUC = {mlp_auc:.3f})')
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {rf_auc:.3f})')
plt.plot(fpr_gb, tpr_gb, label=f'Gradient Boosting (AUC = {gb_auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=0.8)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### Entrenamos modelos generativos

#### Variational Autoencoder (VAE)

In [ ]:
# EXPERIMENTO [ NO BORRAR ]

# Preparo los espectrogramas para enchufarselos al vae
#   - grafico 5 muestras pre y post normalizar
print(audio_df)

original_spec_samples = []
normalized_samples = []
denormalized_samples = []
 
global_min, global_max = compute_global_min_max(audio_df)

whale_audio_samples = audio_df[audio_df['label'] == 1].sample(5)['audio']

for sample in whale_audio_samples:
    mel_spec = get_melspectrogram(sample)
    original_spec_samples.append(mel_spec)

    normalized_mel_spec = normalize_spectrogram(mel_spec, global_min, global_max)
    normalized_samples.append(normalized_mel_spec)

    denormalized_mel_spec = denormalize_spectrogram(normalized_mel_spec, global_min, global_max)
    denormalized_samples.append(denormalized_mel_spec)

fig, axes = plt.subplots(3, 5, figsize=(18, 8))

for i in range(5):
    # Original spectrogram
    img0 = librosa.display.specshow(original_spec_samples[i], sr=SR, hop_length=HOP_LENGTH, ax=axes[0, i], x_axis='time', y_axis='hz', cmap='magma', fmax=MAX_FREQ)
    axes[0, i].set_title(f'Original {i+1}')
    axes[0, i].set_xticks([])
    axes[0, i].set_yticks([])

    # Normalized spectrogram
    img1 = librosa.display.specshow(normalized_samples[i], sr=SR, hop_length=HOP_LENGTH, ax=axes[1, i], x_axis='time', y_axis='hz', cmap='magma', fmax=MAX_FREQ)
    axes[1, i].set_title(f'Normalized {i+1}')
    axes[1, i].set_xticks([])
    axes[1, i].set_yticks([])

    # Denormalized spectrogram
    img2 = librosa.display.specshow(denormalized_samples[i], sr=SR, hop_length=HOP_LENGTH, ax=axes[2, i], x_axis='time', y_axis='hz', cmap='magma', fmax=MAX_FREQ)
    axes[2, i].set_title(f'Denormalized {i+1}')
    axes[2, i].set_xticks([])
    axes[2, i].set_yticks([])

axes[0, 0].set_ylabel('Original')
axes[1, 0].set_ylabel('Normalized')
axes[2, 0].set_ylabel('Denormalized')
plt.tight_layout()
plt.show()

print(original_spec_samples[-1])
print(normalized_samples[-1])
print(denormalized_samples[-1])



In [ ]:
# PASO TODOS LOS AUDIOS A UN ESPECTROGRAMA Y NORMALIZO

mel_specs = []
for whale_audio in audio_df[audio_df['label'] == 1]['audio']:
    mel = get_melspectrogram(whale_audio)
    mel_norm = normalize_spectrogram(mel, global_min, global_max)
    mel_specs.append(mel_norm.flatten())
X = np.array(mel_specs)

In [ ]:
# PREPARO LOS DATOS PARA ENCHUFARSELOS AL VAE

X = np.array(mel_specs)  # shape: (num_samples, N_MELS * time_steps)
X_tensor = torch.tensor(X, dtype=torch.float32)
train_loader = DataLoader(TensorDataset(X_tensor), batch_size=128, shuffle=True)

In [ ]:
# VARIATIONAL AUTOENCODER MODEL

class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim=32):
        super(VAE, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU()
        )

        self.fc_mu = nn.Linear(128, latent_dim)
        self.fc_logvar = nn.Linear(128, latent_dim)

        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, input_dim),
            nn.Sigmoid()
        )

    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        x_hat = self.decode(z)
        return x_hat, mu, logvar

    def loss(self, x, x_hat, mu, logvar):
        recon_loss = nn.functional.mse_loss(x_hat, x, reduction='mean')
        kl_div = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        return recon_loss + kl_div
    
    

In [ ]:
# TRAIN

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vae = VAE(X.shape[1], latent_dim=32).to(device)  # Use all data, not X_train
optimizer = optim.Adam(vae.parameters(), lr=1e-3)

epochs = 50
for epoch in range(epochs):
    vae.train()
    total_loss = 0
    for xb_tuple in train_loader:  # xb_tuple is a tuple (xb,)
        xb = xb_tuple[0].to(device)
        x_hat, mu, logvar = vae(xb)
        loss = vae.loss(xb, x_hat, mu, logvar)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * xb.size(0)  # accumulate per sample
    avg_loss = total_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

In [ ]:
# GENERO MUESTRAS SINTÉTICAS

MAX_FREQ = 600 # ARREGLAR ESTO

vae.eval()
num_samples = 3
latent_dim = 32  # Must match your VAE's latent_dim

with torch.no_grad():
    z = torch.randn(num_samples, latent_dim).to(device)
    generated = vae.decode(z).cpu().numpy()

fig, axes = plt.subplots(1, num_samples, figsize=(12, 4))

for i in range(num_samples):
    # Reshape to (N_MELS, time_steps)
    spec_norm = generated[i].reshape((N_MELS, -1))
    # Denormalize
    spec = denormalize_spectrogram(spec_norm, global_min, global_max)
    librosa.display.specshow(spec, sr=SR, hop_length=HOP_LENGTH, x_axis='time', y_axis='hz', cmap='magma', ax=axes[i], fmax=MAX_FREQ)
    axes[i].set_title(f"Synthetic Sample {i+1}")
    axes[i].set_ylim([0, MAX_FREQ])

plt.tight_layout()
plt.show()

In [ ]:
# ESCUCHAMOS LAS MUESTRAS SINTÉTICAS [ NO PROBE ESTO TODAVIA ]

import librosa
from IPython.display import Audio, display

for i in range(num_samples):
    spec_norm = generated[i].reshape((N_MELS, -1))
    spec = denormalize_spectrogram(spec_norm, global_min, global_max)
    # Invert dB to power
    spec_power = librosa.db_to_power(spec)
    # Invert Mel-spectrogram to waveform
    audio = librosa.feature.inverse.mel_to_audio(
        spec_power, 
        sr=SR, 
        n_fft=N_FFT, 
        hop_length=HOP_LENGTH, 
        n_iter=32, 
        fmax=MAX_FREQ
    )
    print(f"Synthetic Sample {i+1}:")
    display(Audio(audio, rate=SR*1.5))


In [ ]:
# BÚSQUEDA DE HIPERPARÁMETROS [ CAMBIAR PARA QUE SE PUEDA PROBAR DISTINTAS ARQUITECTURAS ]

from sklearn.model_selection import KFold, ParameterGrid

def train_vae(X, latent_dim, lr, epochs, batch_size, device):
    vae = VAE(X.shape[1], latent_dim=latent_dim).to(device)
    optimizer = optim.Adam(vae.parameters(), lr=lr)
    dataset = TensorDataset(torch.tensor(X, dtype=torch.float32))
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    vae.train()
    for epoch in range(epochs):
        for xb_tuple in loader:
            xb = xb_tuple[0].to(device)
            x_hat, mu, logvar = vae(xb)
            loss = vae.loss(xb, x_hat, mu, logvar)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    return vae

def evaluate_vae(vae, X, device):
    vae.eval()
    dataset = TensorDataset(torch.tensor(X, dtype=torch.float32))
    loader = DataLoader(dataset, batch_size=128)
    total_loss = 0
    with torch.no_grad():
        for xb_tuple in loader:
            xb = xb_tuple[0].to(device)
            x_hat, mu, logvar = vae(xb)
            loss = vae.loss(xb, x_hat, mu, logvar)
            total_loss += loss.item() * xb.size(0)
    return total_loss / len(dataset)

# Example grid
param_grid = {
    'latent_dim': [8, 16, 32],
    'lr': [1e-3, 5e-4],
    'epochs': [20, 40],
    'batch_size': [64]
}

kf = KFold(n_splits=3, shuffle=True, random_state=SEED)
results = []

for params in ParameterGrid(param_grid):
    val_losses = []
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        vae = train_vae(X_train, **params, device=device)
        val_loss = evaluate_vae(vae, X_val, device=device)
        val_losses.append(val_loss)
    avg_loss = np.mean(val_losses)
    print(f"Params: {params}, Avg Val Loss: {avg_loss:.4f}")
    results.append((params, avg_loss))

# Find best params
best_params = min(results, key=lambda x: x[1])
print("Best params:", best_params[0])

In [ ]:
# --------------------------
# Visualización con PCA
# --------------------------
vae.eval()
all_mu = []
all_labels = []

with torch.no_grad():
    for xb, yb in val_loader:
        xb = xb.to(device)
        mu, _ = vae.encode(xb)
        all_mu.append(mu.cpu().numpy())
        all_labels.append(yb.numpy())

all_mu = np.concatenate(all_mu, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

pca = PCA(n_components=2)
z_pca = pca.fit_transform(all_mu)

plt.figure(figsize=(8, 6))
scatter = plt.scatter(z_pca[:, 0], z_pca[:, 1], c=all_labels, cmap='coolwarm', alpha=0.7)
plt.title("VAE Latent Space (PCA)")
plt.xlabel("PC 1")
plt.ylabel("PC 2")
plt.colorbar(scatter, label="Clase")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=SEED)
z_tsne = tsne.fit_transform(all_mu)

plt.figure(figsize=(8, 6))
scatter = plt.scatter(z_tsne[:, 0], z_tsne[:, 1], c=all_labels, cmap='coolwarm', alpha=0.7)
plt.title("VAE Latent Space (t-SNE)")
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.colorbar(scatter, label="Clase")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader

# === FUNCIONES PROPIAS REQUERIDAS ===
# - get_melspectrogram(audio)
# - normalize_spectrogram(mel, global_min, global_max)

# === PREPROCESADO ===
mel_specs_ballenas = []
mel_specs_no_ballenas = []

for _, row in audio_df.iterrows():
    mel = get_melspectrogram(row['audio'])  # audio: waveform
    mel_norm = normalize_spectrogram(mel, global_min, global_max)
    mel_flat = mel_norm.flatten()

    if row['label'] == 1:
        mel_specs_ballenas.append(mel_flat)
    else:
        mel_specs_no_ballenas.append(mel_flat)

# Convertimos a arrays
X_ballenas     = np.array(mel_specs_ballenas)  # (N1, N_MELS * T)
X_no_ballenas  = np.array(mel_specs_no_ballenas)  # (N2, N_MELS * T)

# Unimos todo
X_total = np.vstack([X_ballenas, X_no_ballenas])  # (N1+N2, ...)
y_total = np.array([1]*len(X_ballenas) + [0]*len(X_no_ballenas))

# Para PyTorch
X_tensor_total = torch.tensor(X_total, dtype=torch.float32)
y_tensor_total = torch.tensor(y_total, dtype=torch.float32).reshape(-1, 1)

# Para VAE individual (solo ballenas)
X_tensor_ballenas = torch.tensor(X_ballenas, dtype=torch.float32)
train_loader_ballenas = DataLoader(TensorDataset(X_tensor_ballenas), batch_size=128, shuffle=True)

# Para CVAE (todo con labels)
train_loader_total = DataLoader(TensorDataset(X_tensor_total, y_tensor_total), batch_size=128, shuffle=True)


class CVAE(nn.Module):
    def __init__(self, input_dim, latent_dim=32, cond_dim=1):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim + cond_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU()
        )
        self.fc_mu = nn.Linear(128, latent_dim)
        self.fc_logvar = nn.Linear(128, latent_dim)
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim + cond_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, input_dim),
            nn.Sigmoid()
        )

    def encode(self, x, y):
        x_in = torch.cat([x, y], dim=1)
        h = self.encoder(x_in)
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, y):
        z_in = torch.cat([z, y], dim=1)
        return self.decoder(z_in)

    def forward(self, x, y):
        mu, logvar = self.encode(x, y)
        z = self.reparameterize(mu, logvar)
        x_hat = self.decode(z, y)
        return x_hat, mu, logvar

    def loss(self, x, x_hat, mu, logvar):
        recon = nn.functional.mse_loss(x_hat, x, reduction='mean')
        kl = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        return recon + kl

# Preparar datos
X_tensor = torch.tensor(X_total, dtype=torch.float32)
y_tensor = torch.tensor(y_total.reshape(-1, 1), dtype=torch.float32)
dataset = TensorDataset(X_tensor, y_tensor)
loader = DataLoader(dataset, batch_size=128, shuffle=True)

cvae = CVAE(X_total.shape[1], latent_dim=32).to(device)
optimizer = optim.Adam(cvae.parameters(), lr=1e-3)

# Entrenar
for epoch in range(50):
    cvae.train()
    total_loss = 0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        x_hat, mu, logvar = cvae(xb, yb)
        loss = cvae.loss(xb, x_hat, mu, logvar)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * xb.size(0)
    print(f"[CVAE] Epoch {epoch+1}, Loss: {total_loss / len(loader.dataset):.4f}")


vae_ballena = VAE(X_ballenas.shape[1], latent_dim=32).to(device)
vae_noballena = VAE(X_no_ballenas.shape[1], latent_dim=32).to(device)

def train_vae(vae, X_data):
    loader = DataLoader(TensorDataset(torch.tensor(X_data, dtype=torch.float32)), batch_size=128, shuffle=True)
    optim_vae = optim.Adam(vae.parameters(), lr=1e-3)
    for epoch in range(50):
        vae.train()
        total_loss = 0
        for (xb,) in loader:
            xb = xb.to(device)
            x_hat, mu, logvar = vae(xb)
            loss = vae.loss(xb, x_hat, mu, logvar)
            optim_vae.zero_grad()
            loss.backward()
            optim_vae.step()
            total_loss += loss.item() * xb.size(0)
        print(f"[VAE {'ballena' if vae == vae_ballena else 'no-ballena'}] Epoch {epoch+1}, Loss: {total_loss / len(loader.dataset):.4f}")

train_vae(vae_ballena, X_ballenas)
train_vae(vae_noballena, X_no_ballenas)

def get_latent_mu(vae, X, device):
    vae.eval()
    mu_list = []
    loader = DataLoader(torch.tensor(X, dtype=torch.float32), batch_size=128)
    with torch.no_grad():
        for xb in loader:
            xb = xb.to(device)
            mu, _ = vae.encode(xb)
            mu_list.append(mu.cpu().numpy())
    return np.vstack(mu_list)

# CVAE: usar label como condición
X_tensor = torch.tensor(X_total, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y_total.reshape(-1, 1), dtype=torch.float32).to(device)
cvae.eval()
with torch.no_grad():
    mu_cvae, _ = cvae.encode(X_tensor, y_tensor)
    mu_cvae = mu_cvae.cpu().numpy()

# VAE individual
mu_ballena = get_latent_mu(vae_ballena, X_ballenas, device)
mu_noballena = get_latent_mu(vae_noballena, X_no_ballenas, device)

# Unimos y visualizamos
mu_2vae = np.vstack([mu_ballena, mu_noballena])
y_2vae = np.array([1]*len(mu_ballena) + [0]*len(mu_noballena))

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def plot_latent(z, labels, title):
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(z[:, 0], z[:, 1], c=labels, cmap='coolwarm', alpha=0.7)
    plt.colorbar(scatter, label="Clase")
    plt.title(title)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# PCA y t-SNE para ambos métodos
plot_latent(PCA(2).fit_transform(mu_cvae), y_total, "CVAE Latent Space (PCA)")
plot_latent(TSNE(2, random_state=42).fit_transform(mu_cvae), y_total, "CVAE Latent Space (t-SNE)")

plot_latent(PCA(2).fit_transform(mu_2vae), y_2vae, "Dos VAEs Latent Space (PCA)")
plot_latent(TSNE(2, random_state=42).fit_transform(mu_2vae), y_2vae, "Dos VAEs Latent Space (t-SNE)")


In [ ]:
# GAN

import torch
import torch.nn as nn
import torch.optim as optim

# GAN parameters
latent_dim = 32
input_dim = X.shape[1]  # Flattened mel spectrogram size

class Generator(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
            nn.Sigmoid()  # Output normalized [0,1]
        )
    def forward(self, z):
        return self.model(z)

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.model(x)

# Instantiate models
generator = Generator(latent_dim, input_dim).to(device)
discriminator = Discriminator(input_dim).to(device)

# Loss and optimizers
criterion = nn.BCELoss()
lr = 2e-4
g_optimizer = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
d_optimizer = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

# Example training loop (1 epoch)
for epoch in range(epochs):  # Replace with more epochs for real training
    for xb_tuple in train_loader:
        real_data = xb_tuple[0].to(device)
        batch_size = real_data.size(0)

        # Train Discriminator
        z = torch.randn(batch_size, latent_dim, device=device)
        fake_data = generator(z)
        real_labels = torch.ones(batch_size, 1, device=device)
        fake_labels = torch.zeros(batch_size, 1, device=device)

        d_real = discriminator(real_data)
        d_fake = discriminator(fake_data.detach())
        d_loss_real = criterion(d_real, real_labels)
        d_loss_fake = criterion(d_fake, fake_labels)
        d_loss = d_loss_real + d_loss_fake

        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()

        # Train Generator
        z = torch.randn(batch_size, latent_dim, device=device)
        fake_data = generator(z)
        d_fake = discriminator(fake_data)
        g_loss = criterion(d_fake, real_labels)

        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()

    print(f"Epoch {epoch+1} | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f}")